In [9]:
import json

with open('../data/databases/addgene-plasmids-sequences.json') as json_file:
    data = json.load(json_file)

grep -o "https:\/\/media\.addgene\.org\/snapgene-media.*\.gbk" addgene-plasmids-sequences.json > gbk.txt

In [2]:
with open("../data/databases/gbk.txt") as fh:
    gbks=fh.read().splitlines()

In [ ]:
#~195,000
import requests
fails=[]
for i in range(len(gbks)):
    if i%1000:
        print(".",end="")
    try:
        r=requests.get(gbks[i], allow_redirects=True).text
        path=gbks[i].split("/")[-1]
        with open(f'../data/addgene_gbks/{path}', 'w') as fh:
            fh.write(r)
    except:
        fails.append(i)


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [11]:
len(gbks)

195426

In [12]:
len(data['plasmids'])

93823

In [4]:
data['plasmids'][0]['sequences'].keys()

dict_keys(['public_addgene_full_sequences', 'public_addgene_partial_sequences', 'public_user_full_sequences', 'public_user_partial_sequences'])

In [15]:
data['plasmids'][0]['sequences']['public_addgene_partial_sequences']

[{'genbank_url': 'https://media.addgene.org/snapgene-media/v1.6.2-0-g4b4ed87/sequences/64/15/16415/addgene-plasmid-3-sequence-16415.gbk',
  'sequence': 'GGTCGGATTCCCGGGCTGCAGGAATTCGGCACGAGGTTGCCTGCATGAGTGTGTGCTGTGTGTCAGAGTGGATTGGAGTTGAAAAAGCTTGACTGGCGTCATTCGGGAGCTGGATGGCTTGGGACATGTGCAGCCAAGACTCTGTATGGAGTGACATAGAGTGTGCTGCTCTGGTTGGTGAGGACCAGCCTCTTTGCCCAGATCTTCCTGAACTTGACCTTTCTGAACTTGATGTGAATGACTTGGATACAGACAGCTTTCTGGGTGGATTGAAGTGGTGTAGCGACCAATCGGAAATCATATCCAACCAGTACAACAATGAGCCTGCGAACATATTTGAGAAGATAGATGAAGAGAATGAGGCAAACTTGCTAGCGGTCCTCACAGAGACACTGGACAGTCTCCCCGTGGATGAAGACGGATTGCCCTCATTTGATGCACTGACAGATGGAGCCGTGACCACTGACAACGAGGCCAGTCCTTCCTCCATGCCTGACGGCACCCCTCCCCCTCAGGAGGCAGAAGAGCCGTCTCTACTTAAGAAGCTCTTACTGGCACCAGCCAACACTCAGCTCAGCTACAATGAATGCAGCGGTCTTAGCACTCAGAACCATGCAGCAAACCACACCCACAGGATCAGAACAAACCCTGCCATTGTTAAGACCGAGAATTCATGGAGCAATAAAGCGAAGAGCATTTGTCAACAGCAAAAGCCACAAAGACGTCCCTGCTCAGAGCTTCTCAAGTATCTGACCACAAACGATGACCCTCCTCACACCAAACCCACAGAAAACAGGAACAGCAGCAGAGACAAATGTGCTTCCAAAAAGAAGTCCCATACACAACC

In [11]:
from collections import Counter
Counter([len(data['plasmids'][i]['sequences']['public_addgene_partial_sequences']) for i in range(len(data['plasmids']))]).most_common()

[(0, 40344),
 (1, 28137),
 (2, 19410),
 (3, 4269),
 (4, 1190),
 (5, 323),
 (6, 82),
 (7, 38),
 (8, 15),
 (9, 7),
 (10, 6),
 (11, 2)]

In [12]:
Counter([len(data['plasmids'][i]['sequences']['public_addgene_full_sequences']) for i in range(len(data['plasmids']))]).most_common()

[(0, 48114), (1, 45411), (2, 290), (3, 7), (4, 1)]

In [13]:
Counter([len(data['plasmids'][i]['sequences']['public_user_partial_sequences']) for i in range(len(data['plasmids']))]).most_common()

[(0, 68137), (1, 22881), (2, 2531), (3, 149), (4, 107), (5, 17), (6, 1)]

In [14]:
Counter([len(data['plasmids'][i]['sequences']['public_user_full_sequences']) for i in range(len(data['plasmids']))]).most_common()

[(0, 60571), (1, 33158), (2, 94)]

# combing through gbks

In [ ]:
import glob
from Bio import SeqIO
import pandas as pd

from Bio import BiopythonParserWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', BiopythonParserWarning)
    breaker=0
    l=[]
    for file_loc in glob.glob("../data/addgene_gbks/*.gbk"):
        if breaker == None:
            break
        with open(file_loc,'r') as file_handle:
            record_dict = SeqIO.to_dict(SeqIO.parse(file_handle, 'gb'))

        gbkFile = record_dict[list(record_dict.keys())[0]]

        name  = file_loc.split("/")[-1].replace(".gbk","")
        seq   = str(gbkFile.seq)
        description = gbkFile.description
        base = {"name":name,"full_seq":seq,"description":description,"description":description}
        annos = gbkFile.annotations
        l.append({**base,**annos})

        breaker+=1
l=pd.DataFrame(l)

In [23]:
l.to_csv("/Users/mattmcguffie/Desktop/addgene_gbk_metadata.csv",index=["name"])

In [17]:
from Bio.Seq import Seq
l=pd.read_csv("/Users/mattmcguffie/Desktop/addgene_features_from_gbk.csv",index_col=0)
l['rev_seq']=l['seq'].apply(Seq).apply(Seq.reverse_complement).apply(str)
l['canon_seq']=l['seq'].where(l['seq']<=l['rev_seq'],l['rev_seq'])
l.head()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,name,type,seq,gene,label,note,regulatory_class,codon_start,product,translation,...,mobile_element_type,ncRNA_class,estimated_length,db_xref,protein_id,function,allele,seq_len,rev_seq,canon_seq
0,_11_17_51117_addgene-plasmid-39353-sequence-51117,terminator,CCGGTAGAGGTGTGGTCAATAAGAGCGACCTCATACTATACCTGAG...,S. cerevisiae ADH1,ADH1 terminator,transcription terminator for the S. cerevisiae...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,188,GCGAATTTCTTATGATTTATGATTTTTATTATTAAATAAGTTATAA...,CCGGTAGAGGTGTGGTCAATAAGAGCGACCTCATACTATACCTGAG...
6,_13_53_81353_addgene-plasmid-51658-sequence-81353,promoter,CCTATAGTGAGTCGTATTA,NaN,T7 promoter,promoter for bacteriophage T7 RNA polymerase,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,TAATACGACTCACTATAGG,CCTATAGTGAGTCGTATTA
15,_27_81_232781_addgene-plasmid-120241-sequence-...,promoter,GGGCAGAGCGCACATCGCCCACAGTCCCCGAGAAGTTGGGGGGAGG...,NaN,EF-1-alpha core promoter,core promoter for human elongation factor EF-1...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212,CTGTGTTCTGGCGGCAAACCCGTTGCGAAAAAGAACGTTCACGGCG...,CTGTGTTCTGGCGGCAAACCCGTTGCGAAAAAGAACGTTCACGGCG...
16,_27_81_232781_addgene-plasmid-120241-sequence-...,intron,GTAAGTGCCGTGTGTGGTTCCCGCGGGCCTGGCCTCTTTACGGGTT...,NaN,EF-1-alpha intron A,intron upstream of the start codon of human EF...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,939,CTGAAATGGAAGAAAAAAACTTTGAACCACTGTCTGAGGCTTGAGA...,CTGAAATGGAAGAAAAAAACTTTGAACCACTGTCTGAGGCTTGAGA...
18,_27_81_232781_addgene-plasmid-120241-sequence-...,regulatory,GCCGCCATGG,NaN,NaN,vertebrate consensus sequence for strong initi...,other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,CCATGGCGGC,CCATGGCGGC


In [10]:
from collections import Counter
Counter(l['label']).most_common()

[('ori', 77879),
 ('AmpR promoter', 64009),
 (nan, 62074),
 ('AmpR', 58760),
 ('lac operator', 52956),
 ('T7 promoter', 45473),
 ('lac promoter', 41390),
 ('CAP binding site', 39954),
 ('SV40 poly(A) signal', 36788),
 ('f1 ori', 36125),
 ('CMV enhancer', 28008),
 ('SV40 ori', 24568),
 ('CMV promoter', 24345),
 ('6xHis', 22686),
 ('SV40 promoter', 18621),
 ('SV40 NLS', 17673),
 ('tet operator', 15991),
 ('T3 promoter', 15837),
 ('Factor Xa site', 15616),
 ('bGH poly(A) signal', 15129),
 ('cPPT/CTS', 14148),
 ('NeoR/KanR', 14082),
 ('rrnB T1 terminator', 13299),
 ("5' LTR (truncated)", 12266),
 ('WPRE', 12104),
 ('gRNA scaffold', 11685),
 ('attB1', 11025),
 ('attB2', 10872),
 ('RRE', 10839),
 ('HIV-1 Psi', 10767),
 ('T7 terminator', 10653),
 ('KanR', 9846),
 ('PuroR', 9791),
 ('FLAG', 9446),
 ('EGFP', 8834),
 ('TEV site', 8833),
 ('SP6 promoter', 8760),
 ('AAV2 ITR', 8669),
 ('U6 promoter', 8615),
 ('HA', 8445),
 ('loxP', 8405),
 ("3' LTR (Delta-U3)", 8139),
 ('bom', 7629),
 ('rrnB T2 te

In [82]:
canonicalIndex=[]
import statistics

for label in set(l['label']):
    if str(label) == 'nan':
        continue
    t=list(Counter(l[l['label']==label]['canon_seq']).values())
    if len(t)>1:
        stdev=statistics.stdev(t)
    else:
        stdev=0
    canonicalIndex.append((label,max(t)/sum(t),stdev,len(t)))
canonicalIndex=pd.DataFrame(canonicalIndex,columns=['label','maj_fract','stdev',"num"])

In [86]:
canonicalIndex#.sort_values(by=["maj_fract"],ascending=False).head(60)

,label,maj_fract,stdev,num
0,bom,0.426137,594.341375,53
1,HSP terminator,1.000000,0.000000,1
2,RLuc,1.000000,0.000000,1
3,recA (Delta-LexA) promoter,1.000000,0.000000,1
4,HaloTag(R),0.363636,24.641482,18
...,...,...,...,...
861,EF-1-alpha promoter,0.428073,462.813707,57
862,bidirectional TRE promoter,0.250000,0.000000,4
863,5' LTR (truncated),0.899641,1701.277167,42
864,GST,0.913514,330.286868,26


In [87]:
l

,name,type,seq,gene,label,note,regulatory_class,codon_start,product,translation,...,mobile_element_type,ncRNA_class,estimated_length,db_xref,protein_id,function,allele,seq_len,rev_seq,canon_seq
0,_11_17_51117_addgene-plasmid-39353-sequence-51117,terminator,CCGGTAGAGGTGTGGTCAATAAGAGCGACCTCATACTATACCTGAG...,S. cerevisiae ADH1,ADH1 terminator,transcription terminator for the S. cerevisiae...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,188,GCGAATTTCTTATGATTTATGATTTTTATTATTAAATAAGTTATAA...,CCGGTAGAGGTGTGGTCAATAAGAGCGACCTCATACTATACCTGAG...
6,_13_53_81353_addgene-plasmid-51658-sequence-81353,promoter,CCTATAGTGAGTCGTATTA,NaN,T7 promoter,promoter for bacteriophage T7 RNA polymerase,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,TAATACGACTCACTATAGG,CCTATAGTGAGTCGTATTA
15,_27_81_232781_addgene-plasmid-120241-sequence-...,promoter,GGGCAGAGCGCACATCGCCCACAGTCCCCGAGAAGTTGGGGGGAGG...,NaN,EF-1-alpha core promoter,core promoter for human elongation factor EF-1...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212,CTGTGTTCTGGCGGCAAACCCGTTGCGAAAAAGAACGTTCACGGCG...,CTGTGTTCTGGCGGCAAACCCGTTGCGAAAAAGAACGTTCACGGCG...
16,_27_81_232781_addgene-plasmid-120241-sequence-...,intron,GTAAGTGCCGTGTGTGGTTCCCGCGGGCCTGGCCTCTTTACGGGTT...,NaN,EF-1-alpha intron A,intron upstream of the start codon of human EF...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,939,CTGAAATGGAAGAAAAAAACTTTGAACCACTGTCTGAGGCTTGAGA...,CTGAAATGGAAGAAAAAAACTTTGAACCACTGTCTGAGGCTTGAGA...
18,_27_81_232781_addgene-plasmid-120241-sequence-...,regulatory,GCCGCCATGG,NaN,NaN,vertebrate consensus sequence for strong initi...,other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,CCATGGCGGC,CCATGGCGGC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427942,_61_37_256137_addgene-plasmid-131123-sequence-...,terminator,TCATGTAATTAGTTATGTCACGCTTACATTCACGCCCTCCCCCCAC...,S. cerevisiae CYC1,CYC1 terminator,transcription terminator for CYC1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248,GCAAATTAAAGCCTTCGAGCGTCCCAAAACCTTCTCAAGCAAGGTT...,GCAAATTAAAGCCTTCGAGCGTCCCAAAACCTTCTCAAGCAAGGTT...
2427945,_61_37_256137_addgene-plasmid-131123-sequence-...,promoter,CCTATAGTGAGTCGTATTA,NaN,T7 promoter,promoter for bacteriophage T7 RNA polymerase,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,TAATACGACTCACTATAGG,CCTATAGTGAGTCGTATTA
2427949,_61_37_256137_addgene-plasmid-131123-sequence-...,rep_origin,ACGCGCCCTGTAGCGGCGCATTAAGCGCGGCGGGTGTGGTGGTTAC...,NaN,f1 ori,f1 bacteriophage origin of replication; arrow ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,456,AAATTGTAAGCGTTAATATTTTGTTAAAATTCGCGTTAAATTTTTG...,AAATTGTAAGCGTTAATATTTTGTTAAAATTCGCGTTAAATTTTTG...
2427952,_61_37_256137_addgene-plasmid-131123-sequence-...,CDS,AGCTTGGCAGCAACAGGACTAGGATGAGTAGCAGCACGTTCCTTAT...,S. cerevisiae URA3,URA3,"yeast auxotrophic marker, counterselectable wi...",NaN,1.0,"orotidine-5'-phosphate decarboxylase, required...",MSKATYKERAATHPSPVAAKLFNIMHEKQTNLCASLDVRTTKELLE...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8797,ATTTAATATCATGCACGAAAAGCAAACAAACTTGTGTGCTTCATTG...,AGCTTGGCAGCAACAGGACTAGGATGAGTAGCAGCACGTTCCTTAT...


In [62]:
print(gbkFile)

ID: .
Name: Exported
Description: Expression of MMLV proviral DNA-mCherry. Use together with pCMV-intron and VSV-G to package MMLV reporter virus
Number of features: 2
/molecule_type=ds-DNA
/topology=linear
/data_file_division=SYN
/date=14-AUG-2018
/accessions=['.']
/keywords=['PNCA-mCherry']
/source=synthetic DNA construct
/organism=synthetic DNA construct
/taxonomy=[]
/references=[Reference(title='Silencing of proviruses in embryonic cells: efficiency, stability and chromatin modifications.', ...), Reference(title='Direct Submission', ...)]
Seq('TTGATGATGGCCATGTTATCCTCCTCGCCCTTGCTCACCATGGCATCCATGCAT...TCG', IUPACAmbiguousDNA())


In [46]:
print(gbkFile.features[1])
start = gbkFile.features[1].location.start
end   = gbkFile.features[1].location.end
seq   = str(gbkFile.seq[start:end])
Type  = gbkFile.features[1].type
quals = dict(gbkFile.features[1].qualifiers)
t={**{"type":Type,"seq":seq},**quals}

for k,v in t.items():
    if type(v) == list:
        if len(v) != 0:
            t[k] = " | ".join(v)
        else:
            t[k] = ""
t

type: CDS
location: [21:39](-)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: label, Value: ['6xHis']
    Key: product, Value: ['6xHis affinity tag']
    Key: translation, Value: ['HHHHHH']



{'type': 'CDS',
 'seq': 'GTGGTGGTGGTGGTGGTG',
 'codon_start': '1',
 'product': '6xHis affinity tag',
 'label': '6xHis',
 'translation': 'HHHHHH'}

In [20]:
import glob
from Bio import SeqIO
import pandas as pd

from Bio import BiopythonParserWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', BiopythonParserWarning)
    breaker=0
    fails=[]
    l=[]
    for file_loc in glob.glob("../data/addgene_gbks/*.gbk"):
            if breaker == None:
                break
            try:
                with open(file_loc,'r') as file_handle:
                    record_dict = SeqIO.to_dict(SeqIO.parse(file_handle, 'gb'))
                gbkFile = record_dict[list(record_dict.keys())[0]]
                name  = file_loc.split("/")[-1].replace(".gbk","")

                for feat in gbkFile.features:
                    Type  = feat.type            
                    start = feat.location.start
                    end   = feat.location.end
                    seq   = str(gbkFile.seq[start:end])

                    if Type=="source" and len(seq)==len(gbkFile):
                        continue

                    quals = dict(feat.qualifiers)
                    featInfo  = {**{"name":name,"type":Type,"seq":seq},**quals}

                    for k,v in featInfo.items():
                        if type(v) == list:
                            if len(v) != 0:
                                featInfo[k] = " | ".join(v)
                            else:
                                featInfo[k] = ""
                    l.append(featInfo)
            except:
                fails.append((file_loc,feat))
            #breaker+=1
        
l=pd.DataFrame(l)
l=l[l['type']!="primer_bind"]
l['seq_len']=l['seq'].str.len()

In [21]:
l['type'].value_counts()

CDS               346403
promoter          294061
protein_bind      172040
rep_origin        147594
misc_feature       81460
polyA_signal       65714
terminator         56523
regulatory         40410
enhancer           28036
LTR                25916
intron             16787
RBS                13509
misc_RNA           13449
repeat_region      11998
polyA_site          6366
ncRNA               1838
mobile_element      1278
oriT                1231
gap                 1004
gene                 765
misc_recomb          704
3'UTR                523
5'UTR                504
misc_signal          162
tRNA                  30
mat_peptide           21
exon                   4
Name: type, dtype: int64

In [23]:
l.to_csv("/Users/mattmcguffie/Desktop/addgene_features_from_gbk.csv")

In [60]:
l['rev_seq']=l['seq'].apply(Seq).apply(Seq.reverse_complement).apply(str)
l['canon_seq']=l['seq'].where(l['seq']<=l['rev_seq'],l['rev_seq'])

In [63]:
unique=l.drop_duplicates(subset=['canon_seq'])
unique

,name,type,seq,gene,label,note,regulatory_class,codon_start,product,translation,...,ncRNA_class,estimated_length,db_xref,protein_id,function,allele,seq_len,rev_seq,rev,canon_seq
0,_11_17_51117_addgene-plasmid-39353-sequence-51117,terminator,CCGGTAGAGGTGTGGTCAATAAGAGCGACCTCATACTATACCTGAG...,S. cerevisiae ADH1,ADH1 terminator,transcription terminator for the S. cerevisiae...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,188,GCGAATTTCTTATGATTTATGATTTTTATTATTAAATAAGTTATAA...,False,CCGGTAGAGGTGTGGTCAATAAGAGCGACCTCATACTATACCTGAG...
6,_13_53_81353_addgene-plasmid-51658-sequence-81353,promoter,CCTATAGTGAGTCGTATTA,NaN,T7 promoter,promoter for bacteriophage T7 RNA polymerase,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19,TAATACGACTCACTATAGG,False,CCTATAGTGAGTCGTATTA
15,_27_81_232781_addgene-plasmid-120241-sequence-...,promoter,GGGCAGAGCGCACATCGCCCACAGTCCCCGAGAAGTTGGGGGGAGG...,NaN,EF-1-alpha core promoter,core promoter for human elongation factor EF-1...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,212,CTGTGTTCTGGCGGCAAACCCGTTGCGAAAAAGAACGTTCACGGCG...,True,CTGTGTTCTGGCGGCAAACCCGTTGCGAAAAAGAACGTTCACGGCG...
16,_27_81_232781_addgene-plasmid-120241-sequence-...,intron,GTAAGTGCCGTGTGTGGTTCCCGCGGGCCTGGCCTCTTTACGGGTT...,NaN,EF-1-alpha intron A,intron upstream of the start codon of human EF...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,939,CTGAAATGGAAGAAAAAAACTTTGAACCACTGTCTGAGGCTTGAGA...,True,CTGAAATGGAAGAAAAAAACTTTGAACCACTGTCTGAGGCTTGAGA...
18,_27_81_232781_addgene-plasmid-120241-sequence-...,regulatory,GCCGCCATGG,NaN,NaN,vertebrate consensus sequence for strong initi...,other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10,CCATGGCGGC,True,CCATGGCGGC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427388,_39_74_213974_addgene-plasmid-113423-sequence-...,enhancer,ATATGCCAAGTACGCCCCCTATTGACGTCAATGACGGTAAATGGCC...,NaN,CMV enhancer,human cytomegalovirus immediate early enhancer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9409,GATACACTTGATGTACTGCCAAGTGGGCAGTTTACCGTAAATACTC...,False,ATATGCCAAGTACGCCCCCTATTGACGTCAATGACGGTAAATGGCC...
2427561,_73_91_297391_addgene-plasmid-153702-sequence-...,regulatory,CACCATGGGGTACATAAATGTTTTCGCCTTTCCATTCACGATCTAT...,NaN,NaN,vertebrate consensus sequence for strong initi...,other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6272,GCGGCCGCTGGCGGGTGTGTCGAGTGGATGGTAGGATCGACAAAGA...,False,CACCATGGGGTACATAAATGTTTTCGCCTTTCCATTCACGATCTAT...
2427590,_47_51_174751_addgene-plasmid-86931-sequence-1...,CDS,GGGGCACAAGCTGGAGTACAACTTCAACAGCCACAACGTCTATATC...,NaN,Clover,NaN,NaN,1,bright green-yellow fluorescent protein derive...,VSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKF...,...,NaN,NaN,NaN,NaN,NaN,NaN,6082,AGGATGTTGCCGTCCTCCTTGAAGTCGATGCCCTTCAGCTCGATGC...,True,AGGATGTTGCCGTCCTCCTTGAAGTCGATGCCCTTCAGCTCGATGC...
2427639,_56_21_295621_addgene-plasmid-152609-sequence-...,regulatory,CACCATGGCCGAAAATGTCACTGGCTTATTTAAGGATTGCAGTAAA...,NaN,NaN,vertebrate consensus sequence for strong initi...,other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8528,GCGGCCGCTGGCGGGTGTGTCGAGTGGATGGTAGGATCGACAAAGA...,False,CACCATGGCCGAAAATGTCACTGGCTTATTTAAGGATTGCAGTAAA...


In [73]:
unique.sort_values(by='seq_len').iloc[-1]['name']

'_95_39_199539_addgene-plasmid-102726-sequence-199539'

In [76]:
with open('../data/addgene_gbks/_95_39_199539_addgene-plasmid-102726-sequence-199539.gbk','r') as file_handle:
    record_dict = SeqIO.to_dict(SeqIO.parse(file_handle, 'gb'))
gbkFile = record_dict[list(record_dict.keys())[0]]

/usr/local/lib/python3.7/site-packages/Bio/GenBank/__init__.py:364: BiopythonParserWarning: Attempting to fix invalid location '38523..67' as it looks like incorrect origin wrapping. Please fix input file, this could have unintended behavior.
  BiopythonParserWarning,


In [81]:
print(gbkFile.features[-1])

type: enhancer
location: join{[38522:38759](+), [0:67](+)}
qualifiers:
    Key: label, Value: ['CMV enhancer']
    Key: note, Value: ['human cytomegalovirus immediate early enhancer']



In [68]:
featureList=[SeqRecord(Seq(ele[0]),str(ele[1]),description="") for ele in list(zip(unique['canon_seq'].to_list(),unique.index.to_list()))]
SeqIO.write(featureList,'/Users/mattmcguffie/Desktop/unique_features.fasta', 'fasta')

22987

In [67]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
featureList=[SeqRecord(Seq(ele[0]),str(ele[1]),description="") for ele in list(zip(l['seq'].to_list(),l.index.to_list()))]
SeqIO.write(featureList,'/Users/mattmcguffie/Desktop/features.fasta', 'fasta')

1328330